# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlite3 import connect
from sklearn.metrics import classification_report

import sys
import os
import re
import pickle

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

pd.set_option('display.max_columns', None)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
engine = create_engine('sqlite:///InsertDatabaseName.db')

In [3]:
engine.table_names()

['InsertTableName']

In [4]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("InsertTableName",engine)
#df['related'] = df['related'].replace(2, 1)
df['related'] = df['related'].astype('str').str.replace('2', '1')
df['related'] = df['related'].astype('int')
df = df.drop(['child_alone'], axis=1)

In [5]:
X = df.message
Y = df.iloc[:, 4:]
#Can also write Y as df[df.columns[4:]]

In [6]:
Y.shape

(26216, 35)

In [7]:
df['related'].unique()

array([1, 0])

### 2. Write a tokenization function to process your text data

We want to lowercase the text and remove punctuation. Also drop the ID column.

In [8]:
def tokenize(text):
    
    #make the text lowercase
    text = text.lower()
    
    #remove punctuation from the text
    text = text.replace(r'[^\w\s]','')
    
    #stop_words = set(stopwords.words('english'))
    
    #tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        #lemmatize, normalize case, remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
def model_pipeline():

    pipeline = Pipeline([

        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
y_labels = Y.columns

In [11]:
y_labels.values

array(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related',
       'transport', 'buildings', 'electricity', 'tools', 'hospitals',
       'shops', 'aid_centers', 'other_infrastructure', 'weather_related',
       'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'], dtype=object)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [13]:
model = model_pipeline()
fit_model = model.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
y_pred_train = fit_model.predict(X_train)
y_pred_test = fit_model.predict(X_test)
#y_pred = pd.DataFrame(y_pred, columns=y_labels)

In [15]:
print(classification_report(y_test.values, y_pred_test, target_names=y_labels.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5017
               request       0.85      0.39      0.53      1108
                 offer       0.00      0.00      0.00        37
           aid_related       0.76      0.51      0.61      2737
          medical_help       0.57      0.07      0.13       535
      medical_products       0.66      0.07      0.12       352
     search_and_rescue       0.40      0.01      0.02       204
              security       0.00      0.00      0.00       130
              military       0.57      0.05      0.10       223
                 water       0.91      0.18      0.30       392
                  food       0.84      0.37      0.52       746
               shelter       0.79      0.20      0.32       572
              clothing       0.82      0.10      0.18        90
                 money       1.00      0.02      0.05       165
        missing_people       0.31      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
print(classification_report(y_train.values, y_pred_train, target_names=y_labels.values))

                        precision    recall  f1-score   support

               related       0.99      1.00      0.99     15077
               request       1.00      0.93      0.96      3366
                 offer       1.00      0.70      0.83        81
           aid_related       1.00      0.97      0.98      8123
          medical_help       1.00      0.87      0.93      1549
      medical_products       1.00      0.82      0.90       961
     search_and_rescue       1.00      0.77      0.87       520
              security       1.00      0.72      0.84       341
              military       1.00      0.85      0.92       637
                 water       1.00      0.88      0.94      1280
                  food       1.00      0.95      0.97      2177
               shelter       1.00      0.91      0.95      1742
              clothing       1.00      0.83      0.91       315
                 money       1.00      0.83      0.91       439
        missing_people       0.99      

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
fit_model.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fbd8ce1a400>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [18]:
parameters = [
    {
        'clf__estimator__n_estimators': [10, 20],
        'vect__max_df': (0.5, 1.0),
        #'clf__estimator__criterion': ('gini', 'entropy'),
        #'clf__estimator__min_samples_split': (2, 3)
    }
]

cv = GridSearchCV(fit_model, param_grid=parameters, scoring='f1_micro', n_jobs=4, verbose=2)

In [19]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__n_estimators=10, vect__max_df=0.5 ...............
[CV] clf__estimator__n_estimators=10, vect__max_df=0.5 ...............
[CV] clf__estimator__n_estimators=10, vect__max_df=0.5 ...............
[CV] clf__estimator__n_estimators=10, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=10, vect__max_df=1.0, total= 3.6min
[CV] clf__estimator__n_estimators=10, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=10, vect__max_df=0.5, total= 3.6min
[CV] clf__estimator__n_estimators=10, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=10, vect__max_df=0.5, total= 3.7min
[CV] clf__estimator__n_estimators=20, vect__max_df=0.5 ...............
[CV]  clf__estimator__n_estimators=10, vect__max_df=0.5, total= 3.8min
[CV] clf__estimator__n_estimators=20, vect__max_df=0.5 ...............
[CV]  clf__estimator__n_estimators=10, vect__max_df=1.0, total= 3.3min
[CV] clf__estimat

[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 16.7min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 16.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid=[{'clf__estimator__n_estimators': [10, 20], 'vect__max_df': (0.5, 1.0)}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
y_pred_train = cv.predict(X_train)
y_pred_test = cv.predict(X_test)

In [21]:
print(classification_report(y_test.values, y_pred_test, target_names=y_labels))

                        precision    recall  f1-score   support

               related       0.82      0.96      0.88      5017
               request       0.86      0.42      0.56      1108
                 offer       0.00      0.00      0.00        37
           aid_related       0.77      0.59      0.67      2737
          medical_help       0.64      0.07      0.13       535
      medical_products       0.64      0.04      0.07       352
     search_and_rescue       0.56      0.02      0.05       204
              security       1.00      0.01      0.02       130
              military       0.68      0.06      0.11       223
                 water       0.86      0.36      0.51       392
                  food       0.84      0.39      0.53       746
               shelter       0.88      0.24      0.38       572
              clothing       0.78      0.08      0.14        90
                 money       1.00      0.03      0.06       165
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [22]:
print(classification_report(y_train.values, y_pred_train, target_names=y_labels.values))

                        precision    recall  f1-score   support

               related       1.00      1.00      1.00     15077
               request       1.00      0.98      0.99      3366
                 offer       1.00      0.75      0.86        81
           aid_related       1.00      0.99      1.00      8123
          medical_help       1.00      0.93      0.96      1549
      medical_products       1.00      0.93      0.96       961
     search_and_rescue       1.00      0.90      0.95       520
              security       1.00      0.82      0.90       341
              military       1.00      0.92      0.96       637
                 water       1.00      0.97      0.99      1280
                  food       1.00      0.99      0.99      2177
               shelter       1.00      0.96      0.98      1742
              clothing       1.00      0.95      0.97       315
                 money       1.00      0.92      0.96       439
        missing_people       1.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [23]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(tokenize(sentence))
            
            # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':

                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)

        return pd.DataFrame(X_tagged)

In [24]:
def model_pipeline2():
    pipeline = Pipeline([
        
            ('features', FeatureUnion([

                ('text_pipeline', Pipeline([
                    ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf_transformer', TfidfTransformer())
                ])),

                ('starting_verb_transformer', StartingVerbExtractor())
            ])),
        
            ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])
    return pipeline

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [26]:
model = model_pipeline2()
fit_model2 = model.fit(X_train, y_train)

In [27]:
y_pred_train = fit_model2.predict(X_train)
y_pred_test = fit_model2.predict(X_test)

In [28]:
print(classification_report(y_test.values, y_pred_test, target_names=y_labels))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5032
               request       0.77      0.55      0.64      1112
                 offer       0.00      0.00      0.00        33
           aid_related       0.76      0.59      0.66      2685
          medical_help       0.53      0.25      0.34       480
      medical_products       0.69      0.35      0.46       329
     search_and_rescue       0.60      0.21      0.31       173
              security       0.28      0.07      0.11       106
              military       0.62      0.33      0.43       230
                 water       0.76      0.66      0.71       434
                  food       0.80      0.70      0.75       724
               shelter       0.80      0.55      0.65       565
              clothing       0.65      0.55      0.60        97
                 money       0.52      0.29      0.38       170
        missing_people       0.73      

In [29]:
print(classification_report(y_train.values, y_pred_train, target_names=y_labels))

                        precision    recall  f1-score   support

               related       0.83      0.95      0.88     15062
               request       0.79      0.53      0.64      3362
                 offer       0.38      0.09      0.15        85
           aid_related       0.78      0.60      0.68      8175
          medical_help       0.67      0.30      0.41      1604
      medical_products       0.69      0.33      0.45       984
     search_and_rescue       0.68      0.22      0.33       551
              security       0.50      0.07      0.13       365
              military       0.70      0.37      0.49       630
                 water       0.77      0.66      0.71      1238
                  food       0.82      0.70      0.76      2199
               shelter       0.80      0.55      0.65      1749
              clothing       0.77      0.48      0.59       308
                 money       0.65      0.33      0.44       434
        missing_people       0.75      

### 9. Export your model as a pickle file

In [30]:
model_pkl = pickle.dumps('classifier.pkl')

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.